<a href="https://colab.research.google.com/github/Berkay86/DeepPerson/blob/main/POLARITY%20SENTIMENT%20ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
#Kütüphanelerimizi içeriye alalım.
#Veri İşlemleri
!pip3 install snscrape
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string


In [49]:
import re 
import textblob
from textblob import TextBlob

In [50]:
from wordcloud.wordcloud import WordCloud,STOPWORDS
!pip install emot
from emot.emo_unicode import UNICODE_EMOJI
lemmatizer=WordNetLemmatizer()
from wordcloud import ImageColorGenerator
from PIL import Image
import warnings
%matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Using matplotlib backend: agg


In [46]:
query="(worldcup)"
tweets=[]
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
  if i>20000:
    break
  else:
        tweets.append([tweet.date,tweet.id,tweet.url,tweet.user.username,tweet.sourceLabel,tweet.user.location,tweet.content,tweet.likeCount,tweet.retweetCount])
df=pd.DataFrame(tweets,columns=["Date","ID","url","username","source","location","tweet","num_of_likes", "num_of_retweet"])
df.to_csv("sentiment.csv",encoding='utf-8')
df

,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet
0,2022-12-14 12:32:07+00:00,1603004911385460736,https://twitter.com/anas_solhi/status/16030049...,anas_solhi,Twitter for iPhone,,@ZAboukhlal @EnMaroc @FIFAWorldCup Staghfur la...,0,0
1,2022-12-14 12:32:07+00:00,1603004910664138752,https://twitter.com/TheSun/status/160300491066...,TheSun,Echobox,London,Play our World Cup score predictor for a chanc...,0,0
2,2022-12-14 12:32:05+00:00,1603004902934204416,https://twitter.com/T_bop_/status/160300490293...,T_bop_,Twitter for iPhone,"Victoria, Australia",@Abodeable @FIFAWorldCup I saw France / Mbappe,0,0
3,2022-12-14 12:32:05+00:00,1603004900988051456,https://twitter.com/CobraChopzz/status/1603004...,CobraChopzz,Twitter for Android,,@FIFAWorldCup So Morocco to win and Giroud to ...,0,0
4,2022-12-14 12:32:05+00:00,1603004900388270081,https://twitter.com/MehtooCozy/status/16030049...,MehtooCozy,Twitter Web App,Ohio,@FIFAWorldCup First Team Morocco\n\nFirst scor...,0,0
...,...,...,...,...,...,...,...,...,...
19996,2022-12-14 08:52:35+00:00,1602949661555392517,https://twitter.com/UsaJaun/status/16029496615...,UsaJaun,Jetpack.com,,Argentina manager’s public plea to Lionel Mess...,1,0
19997,2022-12-14 08:52:35+00:00,1602949661039374336,https://twitter.com/sawpeter16/status/16029496...,sawpeter16,Twitter for Android,,@binary_x @FIFAWorldCup france 3 0 morocco,0,0
19998,2022-12-14 08:52:34+00:00,1602949658422104064,https://twitter.com/SyaifulIkhwan_/status/1602...,SyaifulIkhwan_,Twitter for Android,,@Carra23 @England If Johnny English as @Englan...,0,0
19999,2022-12-14 08:52:33+00:00,1602949655129636864,https://twitter.com/5dotHayinbanki/status/1602...,5dotHayinbanki,Twitter for Android,,@daily_trust Can't say but Morocco is the most...,1,0


In [51]:
nltk.download('stopwords')
turk_stop_words=list(stopwords.words("turkish"))
emoji=list(UNICODE_EMOJI.keys())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
df["location"]=df["location"].fillna("Unknown")

In [53]:
def ProcessedTweets(text):
  text=text.lower()
  text=" ".join(re.sub("(@[A-Za-z0-9]+)|(^0-9A-Za-z \t]) |(\w+:\/\/\S+)"," ",text).split()) 
  text=re.sub(r'\@\w+|\#\w+|\d+', "", text)
  punct=str.maketrans("","",string.punctuation+string.digits)
  text=text.translate(punct)
  tokens=word_tokenize(text)
  filtered_words=[w for w in tokens if w not in turk_stop_words]
  filtered_words=[w for w in filtered_words if w not in emoji]
  lemmatizer=WordNetLemmatizer()
  lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
  text=" ".join(lemma_words)
  return text



In [54]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
df["Processed_Tweets"]=df["tweet"].apply(ProcessedTweets)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [55]:
df["Processed_Tweets"]

0                               staghfur laah staghfur lah
1        play our world cup score predictor for a chanc...
2                                      i saw france mbappe
3        so morocco to win and giroud to score first le...
4                   first team morocco first scorer boufal
                               ...                        
19996    argentina manager ’ s public plea to lionel me...
19997                                     x france morocco
19998    if johnny english a manager maybe world cup wi...
19999    trust cant say but morocco is the most luckies...
20000    from ashaiman ghana to perform at fifa world c...
Name: Processed_Tweets, Length: 20001, dtype: object

In [56]:
def polarity(tweet):
  return TextBlob(tweet).sentiment.polarity
def sentimenttextblob(polarity):
  if polarity<0:
    return "Negative"
  elif polarity>0:
    return "Positive"


In [57]:
df["Polarity"]=df["Processed_Tweets"].apply(polarity)
df["Sentiment"]=df["Polarity"].apply(sentimenttextblob)
sent=df["Sentiment"].value_counts()
sent
df.to_csv("sentiment.csv",encoding='utf-8')